# CBMPy Tutorial S01 working with SBML Groups

In this section I demonstrate the use of the SBML Level 3 Groups package with CBMPy. Note that, currently, for Groups support the `libsbml-experimental` bindings are required. For more information please see the CBMPy reference guide (available from http://cbmpy.sourceforge.net).

Additional files needed for this tutorial: None.

As always we start by importing CBMPy

In [1]:
import cbmpy


*****
Using CPLEX
*****



C:\Anaconda\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters



CBMPy environment
******************
Revision: r637


***********************************************************************
* Welcome to CBMPy (0.7.20) - PySCeS Constraint Based Modelling       *
*                http://cbmpy.sourceforge.net                         *
* Copyright(C) Brett G. Olivier 2014 - 2017                           *
* Dept. of Systems Bioinformatics                                     *
* Vrije Universiteit Amsterdam, Amsterdam, The Netherlands            *
* CBMPy is developed as part of the BeBasic MetaToolKit Project       *
* Distributed under the GNU GPL v 3.0 licence, see                    *
* LICENCE (supplied with this release) for details                    *
***********************************************************************



To begin let's load one of the small built-in models supplied with CBMPy: `cbmpy_test_core` and assign it to the variable `cmod`.

In [2]:
cmod = cbmpy.readSBML3FBC('cbmpy_test_core')

core_memesa_model.l3.xml
FBC version: 1
M.getNumReactions: 26
M.getNumSpecies: 22
FBC.getNumObjectives: 1
FBC.getNumGeneAssociations: 0
FBC.getNumFluxBounds: 52
Zero dimension compartment detected: Cell
INFO: Active objective: objMaxJ25
Adding objective: objMaxJ25

SBML3 load time: 0.021

INFO: no standard gene encoding detected, attempting to load from annotations.
INFO: used key(s) '[]'
INFO: Added 0 new genes and 0 associations to model


Let's take a closer look at this model.

In [3]:
print(cmod.getReactionIds())

['R01', 'R10', 'R03', 'R02', 'R05', 'R04', 'R16', 'R17', 'R14', 'R15', 'R12', 'R13', 'R07', 'R06', 'R09', 'R08', 'R11', 'R18', 'R19', 'R26', 'R25', 'R24', 'R23', 'R22', 'R21', 'R20']


In [4]:
print(cmod.getSpeciesIds())

['A', 'B', 'C', 'D', 'F1', 'F2', 'G', 'H', 'I1', 'I2', 'J1', 'J2', 'K1', 'K2', 'L', 'M', 'N', 'X0', 'X1', 'X3', 'mE1', 'mE2']


Let's also look at the reaction annotation (this was stuff classically, and confusingly, stored in the SBML "notes" element)

In [5]:
print([r.getAnnotations() for r in cmod.reactions])

[{'abbreviation': 'R01', 'subsystem': None}, {'abbreviation': 'R10', 'subsystem': 'C2'}, {'abbreviation': 'R03', 'subsystem': 'C1'}, {'abbreviation': 'R02', 'subsystem': 'C1'}, {'abbreviation': 'R05', 'subsystem': None}, {'abbreviation': 'R04', 'subsystem': 'C1'}, {'abbreviation': 'R16', 'subsystem': 'C3'}, {'abbreviation': 'R17', 'subsystem': 'C3'}, {'abbreviation': 'R14', 'subsystem': 'C3'}, {'abbreviation': 'R15', 'subsystem': 'C3'}, {'abbreviation': 'R12', 'subsystem': None}, {'abbreviation': 'R13', 'subsystem': 'C3'}, {'abbreviation': 'R07', 'subsystem': 'C2'}, {'abbreviation': 'R06', 'subsystem': 'C2'}, {'abbreviation': 'R09', 'subsystem': 'C2'}, {'abbreviation': 'R08', 'subsystem': 'C2'}, {'abbreviation': 'R11', 'subsystem': 'C2'}, {'abbreviation': 'R18', 'subsystem': 'C3'}, {'abbreviation': 'R19', 'subsystem': 'C3'}, {'abbreviation': 'R26', 'subsystem': None}, {'abbreviation': 'R25', 'subsystem': None}, {'abbreviation': 'R24', 'subsystem': 'C4'}, {'abbreviation': 'R23', 'subsys

Note how some reactions have a `subsystem` annotation defined, let's take a look at them and extract the subsystem id's.

In [6]:
subsystems = []
for R in cmod.reactions:
    print(R.getId(), R.getAnnotation('subsystem'))
    if R.getAnnotation('subsystem') != None and R.getAnnotation('subsystem') not in subsystems:
        subsystems.append(R.getAnnotation('subsystem'))
subsystems.sort()
print('subsystems', subsystems)

('R01', None)
('R10', 'C2')
('R03', 'C1')
('R02', 'C1')
('R05', None)
('R04', 'C1')
('R16', 'C3')
('R17', 'C3')
('R14', 'C3')
('R15', 'C3')
('R12', None)
('R13', 'C3')
('R07', 'C2')
('R06', 'C2')
('R09', 'C2')
('R08', 'C2')
('R11', 'C2')
('R18', 'C3')
('R19', 'C3')
('R26', None)
('R25', None)
('R24', 'C4')
('R23', 'C4')
('R22', None)
('R21', 'C3')
('R20', 'C3')
('subsystems', ['C1', 'C2', 'C3', 'C4'])


We would like to convert these subsystems into and SBML Group. In its most basic form a Group is a, possibly, arbitrary collection of objects which can be collectively treated as a single object for annotation purposes, thus Groups do not effect the evaluation of a model but rather the human interpretation of its elements. Further properties of Groups will be shown below.

Let's begin by simply creating four groups.

In [7]:
for s in subsystems:
    cmod.createGroup(s)

print('GroupIds', cmod.getGroupIds())

('GroupIds', ['C1', 'C2', 'C3', 'C4'])


Now that we have our groups we can add reaction objects. Note that in CBMPy the object itself is added to the Group. We begin by adding R23 and R24 to C4 manually.

In [8]:
C4 = cmod.getGroup('C4')

In [9]:
C4.addMember(cmod.getReaction('R23'))
C4.addMember(cmod.getReaction('R24'))
print('C4 member ids', C4.getMemberIDs())

('C4 member ids', ['R23', 'R24'])


Of course this can easily be scripted so we will delete the C4 members and do everything at once.

In [10]:
C4.deleteMember('R23')
C4.deleteMember('R24')
print('C4 member ids', C4.getMemberIDs())

('C4 member ids', [])


In [11]:
for R in cmod.reactions:
    if R.getAnnotation('subsystem') != None:
        print(R.getId(), R.getAnnotation('subsystem'))
        cmod.getGroup(R.getAnnotation('subsystem')).addMember(R)
        R.deleteAnnotation('subsystem')
        

('R10', 'C2')
('R03', 'C1')
('R02', 'C1')
('R04', 'C1')
('R16', 'C3')
('R17', 'C3')
('R14', 'C3')
('R15', 'C3')
('R13', 'C3')
('R07', 'C2')
('R06', 'C2')
('R09', 'C2')
('R08', 'C2')
('R11', 'C2')
('R18', 'C3')
('R19', 'C3')
('R24', 'C4')
('R23', 'C4')
('R21', 'C3')
('R20', 'C3')


In [12]:
for G in cmod.groups:
    print(G.getId(), G.getMemberIDs())

('C1', ['R03', 'R02', 'R04'])
('C2', ['R10', 'R07', 'R06', 'R09', 'R08', 'R11'])
('C3', ['R16', 'R17', 'R14', 'R15', 'R13', 'R18', 'R19', 'R21', 'R20'])
('C4', ['R24', 'R23'])


Excellent. We now have have groups of reactions but how can we label them as being a subsystem, there are various ways, we could use the Group `name` or `notes` field, CBMPy annotation or even use the (systems biology ontology) SBO or GO term to describe the pathway. Let's try a combination of these.

In [13]:
for G in cmod.groups:
    G.setName(G.getId()) # sets the group name to its initial ID
    G.setNotes('subsystem')
    G.setAnnotation('subsystem', True)
    G.setSBOterm('SBO:0000000') # as an example, this is not a meaningful SBO term


What we have been doing here is labelling the group itself, not the individual members of the group (no inheritance). If we would actually like the members of the group to be labelled/annotated using the Group as a superclass then we need to set the member annotation directly. Again using the C4 Group we can either set the `shared` properties directly:

In [14]:
print(C4.getMemberIDs())
C4.setSharedNotes('subsystem')

['R24', 'R23']


In [15]:
print('C4 shared', C4.getSharedNotes())
print('R23 notes', cmod.getReaction('R23').getNotes())
print('R24 notes', cmod.getReaction('R24').getNotes())

('C4 shared', u'subsystem')
('R23 notes', u'\n  <body xmlns:html="http://www.w3.org/1999/xhtml">\n    <html:p>Abbreviation: R23</html:p>\n    <html:p>SUBSYSTEM: C4</html:p>\n  </body>\n')
('R24 notes', u'\n  <body xmlns:html="http://www.w3.org/1999/xhtml">\n    <html:p>Abbreviation: R24</html:p>\n    <html:p>SUBSYSTEM: C4</html:p>\n  </body>\n')


Currently the new `notes` annotation is an annotation to the group's list of members. This is the normal Groups behaviour, in addition, CBMPy provides functionality that allows one to push the annotation to the group's member objects. In this case not overwriting any existing `notes` (if they exist), only adding them to members where the attribute is undefined (set with the flag `overwrite=False`).

In [16]:
C4.assignSharedNotesToMembers(overwrite=False)
print('C4 shared', C4.getSharedNotes())
print('R23 notes', cmod.getReaction('R23').getNotes())
print('R24 notes', cmod.getReaction('R24').getNotes())

INFO: Assigning shared Notes to members, this cannot be undone.
('C4 shared', u'subsystem')
('R23 notes', u'\n  <body xmlns:html="http://www.w3.org/1999/xhtml">\n    <html:p>Abbreviation: R23</html:p>\n    <html:p>SUBSYSTEM: C4</html:p>\n  </body>\n')
('R24 notes', u'\n  <body xmlns:html="http://www.w3.org/1999/xhtml">\n    <html:p>Abbreviation: R24</html:p>\n    <html:p>SUBSYSTEM: C4</html:p>\n  </body>\n')


Similarly annotations, MIRIAM annotations and SBO terms can be pushed to members, see the group *assignX* methods for more details. Now we can try add an SBO term to the C1 group *without* pushing it to the members.

In [17]:
cmod.getGroup('C1').setSharedSBOterm('SBO:0000000') # SBO:0000000 is an example only

Groups can be defined as either a "collection", "classification" or "partonomy" by default groups are classified as an (arbitrary) collection:

In [18]:
cmod.getGroup('C2').setKind('partonomy')
cmod.getGroup('C3').setKind('classification')

print('C1 kind:', cmod.getGroup('C1').getKind())
print('C2 kind:', cmod.getGroup('C2').getKind())
print('C3 kind:', cmod.getGroup('C3').getKind())

('C1 kind:', 'collection')
('C2 kind:', 'partonomy')
('C3 kind:', 'classification')


Finally, we write the file to an SBML Level 3 FBC version 1 file.

In [19]:
cbmpy.writeSBML3FBC(cmod, 'groups_example.xml', add_groups=True)


INFO: using FBC version: 1
INFO: added 0 non fluxbound parameters to model
Model exported as: groups_example.xml


This basic introduction of the CBMPy Groups functionality should allow you to create custom groups of objects (including Groups). Additional group functionality is conntinuously being implemented, please see the reference guide and the Group object docstrings and methods for more details. For a final test let's load our saved file that includes the groups data.

In [20]:
dmod = cbmpy.readSBML3FBC('groups_example.xml')

FBC version: 1
M.getNumReactions: 26
M.getNumSpecies: 22
FBC.getNumObjectives: 1
FBC.getNumGeneAssociations: 0
FBC.getNumFluxBounds: 52
INFO: Active objective: objMaxJ25
Adding objective: objMaxJ25
Groups support: <GroupsModelPlugin>
Group.getNumGroups: 4

SBML3 load time: 0.021

INFO: no standard gene encoding detected, attempting to load from annotations.
INFO: used key(s) '[]'
INFO: Added 0 new genes and 0 associations to model


In [21]:
print('Group IDs', dmod.getGroupIds())

('Group IDs', ['C1', 'C2', 'C3', 'C4'])
